In [37]:
import h5py
import os
import re
import math
#taking Kuju as example

def seperate_str_byspace(arr, dst=""):
    """ parse list array(list item or values) to string """
    for k in arr:
        if isinstance(k,list):
            dst = dst + " " + seperate_str_byspace(k)
        else:
            dst = dst + " " + str(k)
    return dst.strip()
def find_key_value(key,lines):
    for key_value in lines:
        if str.find(key_value,key) != -1:
            arr = [k.strip() for k in key_value.split(" ") if k != ""]
            value = float(arr[1])
    return value
def calculate_look_ref(er,sh,sr):
    cos_look_ref = (sr**2+(er+sh)**2-er**2)/(2*sr*(er+sh))
    look_ref_rad = math.acos(cos_look_ref)
    look_ref_deg = (look_ref_rad*180)/math.pi
    return look_ref_deg
                                         
#########################################################################################
if __name__ == '__main__':
    # read geometry.h5 file
    geometry = h5py.File('/data2/rsmasdata/lxr/jupyter/insarlab/Kite&Beat/test/geometryRadar.h5','r')
    height = geometry['height'][:]
    incidenceAngle = geometry['incidenceAngle'][:]
    slantRangeDistance = geometry['slantRangeDistance'][:]
    geometry.close
    
    # from ROI_PAC Document
    #cos(look_ref) = [slantRange^2+(earth_radius+spacecraft_height)^2-earth_radius^2]/[2*slantRange*(earth_radius+spacecraft_height)]
    path = os.getenv('SCRATCHDIR')+'/KujuAlosAT422F650/mintpy/inputs/'
    command_info = ['info.py', path+'geometryRadar.h5', '>', path+'info.txt']
    os.system(seperate_str_byspace(command_info))
    f = open(os.getenv('SCRATCHDIR')+'/KujuAlosAT422F650/mintpy/inputs/info.txt', 'r')
    lines = f.readlines()
    f.close()
    # get EARTH_RADIUS value
    earth_radius = find_key_value('EARTH_RADIUS',lines)
    # get HEIGHT value
    spacecraft_height = find_key_value('HEIGHT',lines)
    # get LOOK_REF1/2/3/4
    LOOK_REF1 = find_key_value('LOOK_REF1',lines)    
    LOOK_REF2 = find_key_value('LOOK_REF2',lines)   
    LOOK_REF3 = find_key_value('LOOK_REF3',lines)
    LOOK_REF4 = find_key_value('LOOK_REF4',lines)
    
    # calculate look_ref1/2/3/4 based on ROI_PAC document
    slantRange1 = slantRangeDistance[0,0]
    slantRange2 = slantRangeDistance[0,-1]
    slantRange3 = slantRangeDistance[-1,0]
    slantRange4 = slantRangeDistance[-1,-1]
    Look_ref1 = calculate_look_ref(earth_radius,spacecraft_height,slantRange1)
    Look_ref2 = calculate_look_ref(earth_radius,spacecraft_height,slantRange2)
    Look_ref3 = calculate_look_ref(earth_radius,spacecraft_height,slantRange3)
    Look_ref4 = calculate_look_ref(earth_radius,spacecraft_height,slantRange4)
    
    #compare
    print('LOOK_REF1 in atribute value:')
    print(LOOK_REF1)
    print('Look_ref1 calculted:') 
    print(Look_ref1)
    print('LOOK_REF2 in atribute value:')
    print(LOOK_REF2) 
    print('Look_ref2 calculted:')
    print(Look_ref2)
    print('LOOK_REF3 in atribute value:')
    print(LOOK_REF3)
    print('Look_ref1 calculted:')
    print(Look_ref3)
    print('LOOK_REF4 in atribute value:')
    print(LOOK_REF4)
    print('Look_ref1 calculted:')
    print(Look_ref4)


LOOK_REF1 in atribute value:
32.0834
Look_ref1 calculted:
32.09437017319701
LOOK_REF2 in atribute value:
36.3971
Look_ref2 calculted:
32.85124498324644
LOOK_REF3 in atribute value:
32.0691
Look_ref1 calculted:
32.09437017319701
LOOK_REF4 in atribute value:
36.3851
Look_ref1 calculted:
32.85124498324644


It seems that using atribute value and dataset from geometryRadar.h5 can not get the correct value of LOOK_REF1/2/3/4

Then, I study the code in ROI_PAC module: 'IntSim.f' fortran script. And find the LOOK_REF1/2/3/4 calculation method is:

      sim_corner_range(1) = r0
      sim_corner_range(2) = r0 + (1.d-3 * rsamp) * nrmax
      sim_corner_range(3) = r0
      sim_corner_range(4) = r0 + (1.d-3 * rsamp) * nrmax


ccccc begin loop over simulation corners

      do j = 1 , 4

         time  = sim_corner_time(j)
         range = sim_corner_range(j)


ccccc determine position and velocity of spacecraft at corner point

         call orrmread1(16,time,pos,vel,0)
         call norm(pos,sc_r)
         call norm(vel,sc_vel)
         elp%r_a = r_a
         elp%r_e2 = r_e2
         call latlon(elp,pos,r_llh,i_xyztollh)
         sc_lat = r_llh(1)
         sc_lon = r_llh(2)
         sc_h = r_llh(3)
         call enubasis(sc_lat,sc_lon,enumat)
         call tranmat(enumat,xyz2enu)
         call matvec(enumat,r_bias,r_enubias)
         call matvec(xyz2enu,vel,enuvel)
         sc_hdg = atan2(enuvel(1),enuvel(2))


ccccc solve law of cosines to determine look angle to reference ellipsoid

         elp%r_a = r_a
         elp%r_e2 = r_e2
         peg%r_lat = sc_lat
         peg%r_lon = sc_lon
         peg%r_hdg = sc_hdg+r_sc_az_nom
         img_pln_rad = rdir(r_a,r_e2,sc_hdg+r_sc_az_nom,sc_lat)
         call radar_to_xyz(elp,peg,ptm2)
         call convert_sch_to_xyz(ptm2,sc_sch,pos,i_xyztosch)

         target_d = img_pln_rad
         sc_d = img_pln_rad + sc_sch(3)
         look_angle = acos((sc_d**2 + range**2 - target_d**2) / (2.d0 * sc_d * range)) 

where sc_sch(3)=spacecraft_height, range = slant_range; 

From this script, a conclusion can be drawn that the calculation method in script is same with that in Document. The main difference focus on what's the value of earth_radius. In script earth_radius = img_pln_rad = rdir(r_a,r_e2,sc_hdg+r_sc_az_nom, sc_lat).

Now my question is that I can not find the explantion or content about rdir function in files downloaded from geodesymiami/roipac(Github)...